In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize

nltk.download("punkt")  # Download tokenizer data
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [ ]:
df = pd.read_csv(r"/content/MyDrive/MyDrive/RECIPE NER/full_dataset2.csv", low_memory = False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/MyDrive/MyDrive/RECIPE NER/full_dataset2.csv'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231142 entries, 0 to 2231141
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   title        object
 1   ingredients  object
 2   directions   object
 3   NER          object
dtypes: object(4)
memory usage: 68.1+ MB


In [ ]:
df.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
#df.drop(['Unnamed: 0', "link", "source"], axis=1, inplace=True)

In [ ]:
#df.to_csv("/content/MyDrive/MyDrive/full_dataset2.csv", index = False)

In [ ]:
"""NERs = []
for x in df.NER:
    NERs += eval(x)"""

In [ ]:
#len(NERs)

18921250

In [ ]:
#NERs[100]

'broccoli'

In [ ]:
#NERs = set(NERs)

In [ ]:
#len(NERs)

234059

In [ ]:
"""with open("/content/MyDrive/MyDrive/RECIPE NER/NERs.txt", "w") as f:
    for x in NERs:
        f.write(x + "\n")"""

## RELEVANT KEY WORDS

In [ ]:
fruits_veggies = ['apple', 'pea', 'cucumber', 'chili', 'pepper', 'fig', 'onion', 'jalapeno', 'shallot', 'beet', 'strawberry', 'radish', 'tomato', 'potato', 'lemon', 'pineapple', 'okra', 'grape', 'broccoli', 'zucchini', 'cabbage', 'pear', 'garlic', 'spinach', 'lime', 'eggplant', 'banana', 'mushroom', 'corn', 'fennel', 'carrot', 'lettuce', 'melon']
dairy = ['mozzarella', 'cheese', 'cream', 'ricotta', 'butter', 'cheddar', 'yogurt', 'parmesan', 'milk']
meat_seafood = ['lamb', 'fish', 'chicken', 'lobster', 'duck', 'steak', 'turkey', 'pork', 'mutton', 'rabbit', 'sausage', 'cod', 'crab', 'veal', 'shrimp', 'salmon', 'beef', 'bacon', 'ham']
seasonings = ['tarragon', 'basil', 'bayleaf', 'rosemary', 'nutmeg', 'vanilla', 'oregano', 'clove', 'cinnamon', 'saffron', 'sauce', 'cumin', 'paprika', 'curry', 'vinegar', 'mustard', 'thyme', 'salt', 'anise', 'ginger', 'wine']
nuts_seeds = ['pumpkin', 'pecan', 'hazelnut', 'cashew', 'walnut', 'flax', 'sunflower', 'macadamia', 'almond', 'chia', 'peanut', 'sesame', 'seed', 'pistachio']


In [ ]:
fruits_veggies.extend(['avocado', 'kale', 'celery', 'cauliflower', 'asparagus', 'brussels sprout', 'arugula', 'bell pepper', 'sweet potato', 'yam', 'cherry', 'mango', 'orange', 'kiwi', 'blueberry', 'raspberry', 'blackberry', 'cranberry', 'pomegranate', 'rhubarb', 'squash', 'pumpkin', 'artichoke', 'leek', 'scallion', 'green bean', 'snow pea', 'snap pea', 'collard green', 'turnip', 'rutabaga', 'kohlrabi', 'watercress', 'bok choy', 'napa cabbage', 'radicchio', 'endive', 'grapefruit', 'tangerine', 'clementine', 'plum', 'apricot', 'peach', 'nectarine', 'watermelon', 'cantaloupe', 'honeydew'])
dairy.extend(['mascarpone', 'goat cheese', 'brie', 'camembert', 'feta', 'cottage cheese', 'swiss cheese', 'blue cheese', 'gouda', 'provolone', 'gruyere', 'monterey jack', 'queso fresco', 'cream cheese', 'heavy cream', 'half and half', 'buttermilk', 'sour cream', 'whipped cream', 'evaporated milk', 'condensed milk', 'ghee', 'kefir'])
meat_seafood.extend(['egg', 'eggs', 'tuna', 'tilapia', 'halibut', 'anchovy', 'sardine', 'trout', 'mackerel', 'clam', 'oyster', 'mussel', 'scallop', 'calamari', 'squid', 'octopus', 'prawn', 'venison', 'bison', 'ground beef', 'ground turkey', 'meatball', 'bratwurst', 'chorizo', 'salami', 'pepperoni', 'prosciutto', 'pancetta', 'pastrami', 'corned beef', 'liver', 'kidney', 'oxtail', 'rib', 'tenderloin', 'sirloin', 'flank steak', 'ribeye', 'filet mignon', 'drumstick', 'thigh', 'breast', 'wing'])
seasonings.extend(['pepper', 'cayenne', 'turmeric', 'cardamom', 'cilantro', 'parsley', 'mint', 'dill', 'sage', 'marjoram', 'coriander', 'fennel seed', 'allspice', 'star anise', 'chili powder', 'garlic powder', 'onion powder', 'soy sauce', 'fish sauce', 'hoisin sauce', 'sriracha', 'tabasco', 'worcestershire sauce', 'ketchup', 'mayonnaise', 'aioli', 'pesto', 'chimichurri', 'harissa', 'za\'atar', 'sumac', 'five spice', 'garam masala', 'red wine vinegar', 'white wine vinegar', 'balsamic vinegar', 'apple cider vinegar', 'rice vinegar', 'miso', 'wasabi', 'horseradish', 'capers', 'olive oil', 'sesame oil', 'coconut oil', 'vegetable oil', 'canola oil', 'peanut oil', 'lemon juice', 'lime juice', 'honey', 'maple syrup', 'brown sugar', 'white sugar', 'powdered sugar'])
nuts_seeds.extend(['pine nut', 'brazil nut', 'chestnut', 'poppy seed', 'hemp seed', 'coconut', 'almond butter', 'peanut butter', 'cashew butter', 'tahini', 'pumpkin seed', 'sunflower seed', 'almond meal', 'almond flour', 'ground flaxseed', 'pepita', 'nut milk', 'almond milk', 'cashew milk', 'coconut milk'])
grains_starches = ['rice', 'white rice', 'brown rice', 'jasmine rice', 'basmati rice', 'arborio rice', 'wild rice', 'quinoa', 'couscous', 'barley', 'farro', 'bulgur', 'oats', 'oatmeal', 'pasta', 'spaghetti', 'linguine', 'penne', 'fettuccine', 'macaroni', 'ramen', 'udon', 'noodle', 'bread', 'flour', 'all-purpose flour', 'whole wheat flour', 'bread flour', 'cake flour', 'cornmeal', 'cornstarch', 'potato starch', 'tapioca starch', 'baking powder', 'baking soda', 'yeast', 'cracker', 'tortilla', 'pita', 'baguette', 'sourdough', 'rye', 'brioche', 'semolina', 'polenta', 'grits']
cooking_methods = ['bake', 'boil', 'broil', 'fry', 'grill', 'poach', 'roast', 'saute', 'simmer', 'steam', 'braise', 'blanch', 'caramelize', 'char', 'deglaze', 'dice', 'fold', 'knead', 'marinate', 'mince', 'mix', 'puree', 'reduce', 'render', 'sear', 'slice', 'smoke', 'stir', 'toast', 'whip', 'whisk', 'chop', 'blend', 'pressure cook', 'slow cook', 'deep fry', 'pan fry', 'stir fry', 'air fry', 'sous vide', 'pickle', 'ferment', 'cure', 'candy', 'proof', 'glaze', 'stuff', 'truss', 'spatchcock', 'butterfly', 'pound', 'score', 'zest', 'julienne', 'brunoise', 'chiffonade']
cooking_tools = ['oven', 'stove', 'microwave', 'blender', 'food processor', 'mixer', 'slow cooker', 'pressure cooker', 'air fryer', 'grill', 'frying pan', 'skillet', 'pot', 'saucepan', 'dutch oven', 'baking sheet', 'baking pan', 'loaf pan', 'muffin tin', 'cake pan', 'pie dish', 'casserole dish', 'mixing bowl', 'cutting board', 'knife', 'chef knife', 'paring knife', 'serrated knife', 'whisk', 'spatula', 'spoon', 'ladle', 'tongs', 'grater', 'peeler', 'colander', 'strainer', 'sieve', 'measuring cup', 'measuring spoon', 'scale', 'thermometer', 'timer', 'rolling pin', 'pastry brush', 'mortar and pestle', 'garlic press', 'can opener', 'corkscrew', 'zester', 'mandoline', 'kitchen shears', 'parchment paper', 'foil', 'plastic wrap', 'cheesecloth']
measurements = [
    'teaspoon', 'teaspoons', 'tablespoon', 'tablespoons', 'cup', 'cups', 'pint', 'pints', 'oz',
    'quart', 'quarts', 'gallon', 'gallons', 'ounce', 'ounces', 'pound', 'pounds', 'gram', 'grams',
    'kilogram', 'kilograms', 'milliliter', 'milliliters', 'liter', 'liters', 'pinch', 'pinches',
    'dash', 'dashes', 'handful', 'handfuls', 'sprig', 'sprigs', 'clove', 'cloves', 'head', 'heads',
    'bunch', 'bunches', 'stalk', 'stalks', 'slice', 'slices', 'piece', 'pieces', 'wedge', 'wedges',
    'half', 'halves', 'quarter', 'quarters', 'third', 'thirds', 'whole', 'wholes', 'large', 'larges',
    'medium', 'mediums', 'small', 'smalls', 'tsp', 'tsps', 'tbsp', 'tbsps', 'oz', 'ozs', 'lb', 'lbs',
    'g', 'gs', 'kg', 'kgs', 'ml', 'mls', 'l', 'ls', 'pt', 'pts', 'qt', 'qts', 'gal', 'gals'
]


## ANNOTATION AND DATASET CREATION FOR FINETUNING

In [ ]:
category_keywords = {
    "FRUITS_VEGGIES": fruits_veggies,
    "DAIRY": dairy,
    "MEAT_SEAFOOD" : meat_seafood,
    "SEASONINGS" : seasonings,
    "NUTS_SEEDS" : nuts_seeds,
    "COOKING_METHOD": cooking_methods,
    "COOKING_TOOL": cooking_tools,
    "MEASURE": measurements,
    "GRAINS_STARCH": grains_starches
}

In [ ]:
# Sort multi-word entities by length (longest first to avoid partial matches)
multi_word_entities = sorted(
    [phrase for phrases in category_keywords.values() for phrase in phrases if " " in phrase],
    key=lambda x: len(x.split()),
    reverse=True
)

single_word_entities = {word: category for category, words in category_keywords.items() for word in words if " " not in word}

In [ ]:
# Label mapping for Hugging Face JSON format
label_map = {
    "O": 0,
    "B-FRUITS_VEGGIES": 1, "I-FRUITS_VEGGIES": 2,
    "B-DAIRY": 3, "I-DAIRY": 4,
    "B-MEAT_SEAFOOD": 5, "I-MEAT_SEAFOOD": 6,
    "B-NUTS_SEEDS": 7, "I-NUTS_SEEDS": 8,
    "B-SEASONINGS": 9, "I-SEASONINGS": 10,
    "B-COOKING_METHOD": 11, "I-COOKING_METHOD": 12,
    "B-COOKING_TOOL": 13, "I-COOKING_TOOL": 14,
    "B-MEASURE": 15, "I-MEASURE": 16,
    "B-GRAINS_STARCH": 17, "I-GRAINS_STARCH": 18
}

In [ ]:
"""
# Function to annotate text using NLTK tokenizer
def auto_annotate(text):
    text = text.lower()  # Normalize case
    tokens = word_tokenize(text)  # Tokenize using NLTK
    annotated_tokens = []
    i = 0

    while i < len(tokens):
        matched = False

        # Check for multi-word entities
        for phrase in multi_word_entities:
            phrase_words = phrase.split()
            if tokens[i:i + len(phrase_words)] == phrase_words:
                category = next((cat for cat, vals in category_keywords.items() if phrase in vals), "O")
                annotated_tokens.append(f"{phrase_words[0]} B-{category}")
                for w in phrase_words[1:]:
                    annotated_tokens.append(f"{w} I-{category}")
                i += len(phrase_words)  # Move pointer ahead
                matched = True
                break

        # Check for single-word entities if no multi-word match
        if not matched:
            word = tokens[i]
            category = next((cat for cat, vals in category_keywords.items() if word in vals), "O")
            label = f"B-{category}" if category != "O" else "O"
            annotated_tokens.append(f"{word} {label}")
            i += 1  # Move to next word

    return "\n".join(annotated_tokens)
"""

In [ ]:
# Function to annotate text & return structured output (No JSON writing)
def auto_annotate_to_json(texts):
    dataset = []

    for text in texts:
        text = text.lower()  # Normalize case
        tokens = word_tokenize(text)  # Tokenize using NLTK
        token_labels = ["O"] * len(tokens)  # Default label "O"
        i = 0

        # Process multi-word entities first (longest match priority)
        while i < len(tokens):
            matched = False
            for phrase in multi_word_entities:
                phrase_words = word_tokenize(phrase)
                if tokens[i:i + len(phrase_words)] == phrase_words:
                    category = next((cat for cat, vals in category_keywords.items() if phrase in vals), "O")
                    token_labels[i] = f"B-{category}"
                    for j in range(1, len(phrase_words)):
                        token_labels[i + j] = f"I-{category}"
                    i += len(phrase_words)  # Skip matched words
                    matched = True
                    break

            if not matched:
                i += 1  # Move to next token

        # Process single-word entities (faster lookup)
        for i, token in enumerate(tokens):
            if token in single_word_entities:
                category = single_word_entities[token]
                token_labels[i] = f"B-{category}"

        # Convert labels to numeric format
        numeric_labels = [label_map[label] for label in token_labels]

        # Append in Hugging Face JSON format

        return    {"tokens": tokens, "ner_tags": numeric_labels}



In [ ]:
df.shape

(2231142, 4)

In [ ]:
dfx = df.sample(frac = 0.05) # selects 100K rows randomly 2, 000, 000 x 0.05

In [ ]:
dfx.reset_index(inplace = True)

In [ ]:
dfx.head()

,index,title,ingredients,directions,NER
0,1328821,Cinnamon Crisps,"[""1 Cup Sugar"", ""1 Cup Margarine or Butter"", ""...","[""Mix sugar with butter/margarine until smooth...","[""Sugar"", ""Margarine"", ""cinnamon"", ""flour"", ""E..."
1,1067114,Diner Home Fries From Leftover Baked Potatoes,"[""2 tablespoons vegetable oil"", ""4 potatoes, b...","[""Heat oil in a large frying pan (or use your ...","[""vegetable oil"", ""potatoes"", ""onion"", ""green ..."
2,759999,Best Lasagne,"[""1 lb. ground beef"", ""3 1/2 c. (32 oz. jar) t...","[""Brown beef, drain off fat."", ""Add sauce and ...","[""ground beef"", ""onions"", ""water"", ""eggs"", ""ch..."
3,1055600,Cajun Shrimp Po' Boys,"[""vegetable oil, for frying"", ""2 cups all-purp...","[""Preheat the oven to 350 degrees F."", ""Heat 2...","[""vegetable oil"", ""flour"", ""creole seasoning"",..."
4,1078111,Italian Picnic Salad,"[""1 (14 ounce) can quartered artichoke hearts,...","[""Combine artichoke hearts, green pepper, chee...","[""quartered artichoke hearts"", ""green pepper"",..."


In [ ]:
# Example usage
text = eval(dfx.directions[0])
annotated_text = auto_annotate_to_json(text)
print(annotated_text)

[[{'tokens': ['mix', 'sugar', 'with', 'butter/margarine', 'until', 'smooth', '.', 'add', 'flour/cinnamon', 'mixture', 'and', 'stir', '.']}, {'ner_tags': [11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0]}], [{'tokens': ['spread', 'dough', 'evenly', 'into', 'jelly-roll', 'type', 'pan', '(', 'spread', 'w/fingers', 'and', 'then', 'roll', 'with', 'plain', 'glass', 'tumbler', 'to', 'get', 'the', 'dough', 'even', '.', 'cut', 'with', 'knife', 'into', 'diamond', 'shapes', '.', 'put', 'small', 'drop', 'of', 'egg', 'yolk', 'into', 'center', 'of', 'each', 'diamond', 'and', 'place', '1/2', 'pecan', 'on', 'top', '.', 'bake', 'at', '325', 'for', 'about', '20', 'minutes', '.', 're-cut', 'into', 'diamond', 'shapes', 'while', 'hot', '.', 'remove', 'from', 'pan', 'with', 'a', 'spatula', 'when', 'cook', 'and', 'store', 'in', 'airtight', 'container', '.']}, {'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0,

In [ ]:
train = []
for i in range(len(dfx)):
    if i % 10000 == 0 :
        print(f"{i} rows processed")

    ing = eval(dfx.loc[i, 'ingredients'])
    dir = eval(dfx.loc[i, 'directions'])
    text = ing + dir
    annotated_text = auto_annotate_to_json(text)
    #print(annotated_text)
    train.append(annotated_text)

0 rows processed
10000 rows processed
20000 rows processed
30000 rows processed
40000 rows processed
50000 rows processed
60000 rows processed
70000 rows processed
80000 rows processed
90000 rows processed
100000 rows processed
110000 rows processed


In [ ]:
len(train)

111557

In [ ]:
import pickle

# Save to a file
with open("/content/MyDrive/MyDrive/RECIPE NER/annotated_train_100k.pkl", "wb") as f:
    pickle.dump(train, f)

In [ ]:
trainx = train[:85000]
testx = train[85000:]

In [ ]:
import json
with open("/content/MyDrive/MyDrive/RECIPE NER/annotated_train_100k.json", "w") as f:
    json.dump(trainx, f)

with open("/content/MyDrive/MyDrive/RECIPE NER/annotated_test_100k.json", "w") as f:
    json.dump(testx, f)

######################### RUN FROM HERE

In [16]:
!pip install datasets
from datasets import load_dataset

In [17]:
dataset = load_dataset("json", data_files={"train": "annotated_train_100k.json", "test": "annotated_test_100k.json"})

In [18]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_name = "bert-base-cased"  # Use "bert-large-cased" for better performance
num_labels = 19  # Count unique labels

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,  # ✅ Ensures tokens are truncated to model max length
        padding="max_length",  # ✅ Pads sequences to the max length
        max_length=512,  # ✅ Standard for BERT
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(example["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Maps tokens to words
        label_ids = [-100 if w is None else label[w] for w in word_ids]  # Aligns labels
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
"""
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(example["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if w is None else label[w] for w in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs"""

# Apply to dataset
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/85000 [00:00<?, ? examples/s]

Map:   0%|          | 0/26557 [00:00<?, ? examples/s]

In [22]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification


training_args = TrainingArguments(
    output_dir="./bert-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none"
)

"""trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)"""
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator  # ✅ Fixes inconsistent batch sizes
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.003300,0.007476


TrainOutput(global_step=5313, training_loss=0.01507322732818434, metrics={'train_runtime': 8670.9545, 'train_samples_per_second': 9.803, 'train_steps_per_second': 0.613, 'total_flos': 2.221363794432e+16, 'train_loss': 0.01507322732818434, 'epoch': 1.0})

In [25]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=706d6d4330d14a8cf6b0feadec6295292ebf5b7239cfe9fc74a51abba1c7a6ab
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [26]:
from seqeval.metrics import classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(-1)
    return classification_report(labels, predictions)

trainer.evaluate()

{'eval_loss': 0.007476239465177059,
 'eval_runtime': 829.2547,
 'eval_samples_per_second': 32.025,
 'eval_steps_per_second': 2.002,
 'epoch': 1.0}

In [27]:
model.save_pretrained("./bert-ner-finetuned")
tokenizer.save_pretrained("./bert-ner-finetuned")

('./bert-ner-finetuned/tokenizer_config.json',
 './bert-ner-finetuned/special_tokens_map.json',
 './bert-ner-finetuned/vocab.txt',
 './bert-ner-finetuned/added_tokens.json',
 './bert-ner-finetuned/tokenizer.json')

In [ ]:
# Load later
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("./bert-ner-finetuned")